In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import numpy as np
from tqdm import tqdm
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os
import cv2
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.utils import to_categorical


In [ ]:
DATADIR = '/content/gdrive/MyDrive/Dataset/PetImages'
CATEGORIES = ["Dog","Cat"]
IMG_SIZE = 224

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        count = 0

        for img in tqdm(os.listdir(path), desc=category, total=1000):
            if count >= 1000:
                break
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = tf.cast(cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)), tf.float32)
                training_data.append([new_array, class_num])
                count += 1
                if count%200==0:
                    print(f"Successfully loaded and added {img} to training_data.")
            except Exception as e:
                continue

In [ ]:
create_training_data()
random.shuffle(training_data)


Dog:   8%|▊         | 78/1000 [00:23<05:05,  3.02it/s]

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# convert X and y to numpy arrays
print()
X = np.array(X)
y = np.array(y)

# split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
# print the shape of training and testing sets
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
callback = keras.callbacks.EarlyStopping(monitor = 'accuracy')

model = keras.Sequential([

    keras.layers.Conv2D(64,3, activation = tf.nn.relu, input_shape = (224,224,3)),
    keras.layers.Conv2D(64,3, activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2),

    keras.layers.Conv2D(128,3, activation = tf.nn.relu),
    keras.layers.Conv2D(128,3, activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2),

    keras.layers.Conv2D(256,3, activation = tf.nn.relu),
    keras.layers.Conv2D(256,3, activation = tf.nn.relu),
    keras.layers.Conv2D(256,3, activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2),

    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2),

    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.Conv2D(512,3, activation = tf.nn.relu),
    keras.layers.MaxPooling2D(2),

    keras.layers.Flatten(),

    keras.layers.Dense(4096, activation = tf.nn.relu),
    keras.layers.Dense(4096, activation = tf.nn.relu),
    keras.layers.Dense(1000, activation = tf.nn.relu),

    keras.layers.Dense(2, activation=tf.nn.softmax)
])



model.compile(optimizer= 'adam',#use the Adam optimizer for training the model
              loss='categorical_crossentropy',   # use sparse categorical cross-entropy loss as the loss function
              metrics=['accuracy'])               # use accuracy as the evaluation metric

model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test)) # Train the model on the training data

# Evaluate the model on the test set
#test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the accuracy on the test set
#print("Test accuracy:", test_acc)